In [1]:
%load_ext autoreload
%autoreload 2

In [10]:
import sys
import os
sys.path.append(os.path.abspath(".."))


import torch.nn as nn
from metrics.charts import ChartUtil
from metrics.charts import TrainingCharts
from datasets.xor import XOR
from training.train import TrainUtil

In [ ]:
dataset = XOR()
in_features = 10
out_features = 2
X_train, y_train, X_val, y_val = dataset.get_dataset(in_features = in_features)
trainUtil = TrainUtil(X_train, y_train, X_val = X_val, y_val = y_val)

In [8]:
from layers.customLinearLayer import CustomLinearLayer
from layers.hippocampusLayer import HippocampusLayer
from layers.neoCortexLayer import NeoCortexLayer


class MarrsTwoLayerModel(nn.Module):
    def __init__(self, in_features = in_features, 
                 out_features = out_features,
                 memory_size = 1000,
                 first_layer_out_features = 10,
                 second_layer_out_features = 1000,
                 alpha = 0.02):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.memory_size = memory_size
        self.hippocampal_connections = second_layer_out_features
        self.hippocampal_L_value = int(alpha * second_layer_out_features)
        self.indicator_cells = first_layer_out_features

        self._neo_cortex = NeoCortexLayer(self.in_features,
                                      memory_size = self.memory_size,
                                      out_features = self.indicator_cells,
                                      hippocampol_connections=self.hippocampal_connections,)
        
        self._hippocampus = HippocampusLayer(self.indicator_cells, 
                                            self.hippocampal_connections, 
                                            out_features_allowed= self.hippocampal_L_value)
        
        self._recalled_output = CustomLinearLayer(memory_size, out_features)

    def forward(self, x):
        x_l1 = self._neo_cortex(x)
        x_l2 = self._hippocampus(x_l1)
        x_l3 = self._neo_cortex.coritcal_memory(x_l2)
        out = self._recalled_output(x_l3)
        return out

In [9]:
def perform_one_cortical_experiment(n_epochs = 41, 
                          n_runs=60,
                          in_features = in_features, 
                          out_features = out_features,
                          memory_size = 1000,
                          first_layer_out_features = 10,
                          second_layer_out_features = 1000,
                          alpha = 0.02,
                          trainUtil = trainUtil,
                          print_summary = False):
    chartUtil = ChartUtil()
    epoch_num = n_epochs
    for _ in range(n_runs):
        marrsTwoLayerModel = MarrsTwoLayerModel(
                          in_features = in_features, 
                          out_features = out_features,
                          memory_size = memory_size,
                          first_layer_out_features = first_layer_out_features,
                          second_layer_out_features = second_layer_out_features,
                          alpha = alpha
                          )
        
        loss_over_epochs, accuracy_over_epochs, epoch_numbers, train_accuracy, val_accuracy = trainUtil.train_and_evaluate(marrsTwoLayerModel, 
                                                                                                                           epochs=epoch_num, 
                                                                                                                           name="Marrs Two Layer Model",
                                                                                                                           print_msg=print_summary)
        chartUtil.add_train_data("Marrs Two Layer Model", loss_over_epochs, accuracy_over_epochs, epoch_numbers)
        chartUtil.add_test_data("Marrs Two Layer Model", train_accuracy, val_accuracy)
    
    return chartUtil

NameError: name 'trainUtil' is not defined

In [ ]:

chart_marrs_model = perform_one_cortical_experiment(n_epochs = 200, 
                                                    n_runs= 5, 
                                                    print_summary=True, 
                                                    memory_size = 100,
                                                    first_layer_out_features = 5,
                                                    second_layer_out_features = 50,
                                                    alpha = 0.2)

In [ ]:
training_loss = chart_marrs_model.plot_training_data_for()
training_acc = chart_marrs_model.plot_training_data_for(TrainingCharts.ACC)
test_acc = chart_marrs_model.plot_test_accu_for_models()
training_loss.show()
training_acc.show()
test_acc.show()